# Genetic Algorithm Model

The architecture for the project is as follows:
- Agent class
  - Defined by a base abstract Agent class.
  - You will create individual agents that inherit from this class.
  - The model/organism/subject or "Agent" that will make up our various populations for the algorithm.
  - It has methods to evaluate itself (the fitness function), mutate it's genome for variation, and a static crossover function for combining multiple genomes of a variable number of agents.
- Genetic Algorithm Class
  - A class that orchestrates the three steps of a genetic algorithm generation 1. Populate, 2. Evaluate, 3. Crossover
  - It can take in any Agent that inherits from the Abstract Agent class.
  - Only performs one "generation".
- Main Program
  - Utilizes the Genetic Algorithm and defined Agents to run many generations with specific sized populations and uses data to test the trained agents.

## Agents

Run this cell to import nessecary libraries and dependencies, these are mainly just for typing, nothing important to actual model.

In [ ]:
from abc import ABC, abstractmethod
from typing_extensions import Self
from typing import Callable, List
import copy

### Agent Abstract Class
The base class for an "agent" of our genetic algorithm.

In [ ]:
class Agent(ABC):
  @abstractmethod
  def __init__(self, mutation_rate: float):
    pass

  @abstractmethod
  def challenge(self, *args, **kwargs) -> float:
      pass

  @abstractmethod
  def mutate(self) -> Self:
      pass

  @staticmethod
  @abstractmethod
  def crossover(agents: List) -> Self:
    pass

A child class that inherts and implements from the abstract parent Agent class. This agent performs regression on a function in the XY coordinate space

In [ ]:
from ast import Num
import torch
from torch import nn
from random import gauss, random

class XYAgent(Agent):

  def __init__(self, rate: float):

    self.brain = nn.Sequential(
        nn.Flatten(),
        nn.Linear(2, 20),
        nn.ReLU(),
        nn.Linear(20, 10),
        nn.ReLU(),
        nn.Linear(10, 1)

    )
    self.rate = 0.4

  def mutate(self):
    genome = self.get_genome()

    genome = genome.flatten()
    for i in range(0, len(genome)):
        r = random()
        if r < self.rate:
          genome[i] += gauss(0, 0.5)

    self.set_genome(genome)

    return self


  def get_genome(self):

    genome = torch.tensor([])

    weights = [param for name, param in self.brain.named_parameters() if "weight" in name]
    for weight in weights:
      weight = torch.flatten(weight)
      genome = torch.cat((genome, weight))

    return genome


  def set_genome(self, genome):

    genome_index = 0
    weights = [param for name, param in self.brain.named_parameters() if "weight" in name]

    for w in weights:
      # Calculate the size of the current weight tensor
      num_params = w.numel()

      # Extract the corresponding section of the genome
      w_flat = genome[genome_index:genome_index + num_params]

      # Reshape and assign the weight tensor
      w.data = w_flat.reshape(w.shape).to(w.device)

      # Update the genome index
      genome_index += num_params



  def challenge(self, xs, ys):
    count_correct = 0

    X = torch.tensor(xs).float()
    y = torch.tensor(ys).float()
    y_pred = self.brain(X)

    for i in range(0, len(y_pred)):
      if y_pred[i] == y[i]:
        count_correct += 1

    # Returns total correct classifications of inputs / total number of inputs
    return count_correct / len(xs)


  def crossover(agents):
    # Choose the best agent
    return agents[-1]


## The Genetic Algorithm

In [ ]:
class GeneticAlgorithm:
  def __init__(self):
    self.agents = []

  def populate(self, agent, population_size: int):
    self.agents = [copy.deepcopy(agent).mutate() for _ in range(population_size)]

  def evaluate(self, best: int, challenge_args: list = None) -> List[Agent]:
    if not challenge_args:
      scores = [agent.challenge() for agent in self.agents]
    else:
      scores = [agent.challenge(*challenge_args) for agent in self.agents]

    arranged = [x for _, x in sorted(zip(scores, self.agents), key=lambda t: t[0])]
    return arranged[-best:]

  def crossover(self, agents: List[Agent], cross: Callable[[List[Agent]], Agent]) -> Agent:
    crossed = cross(agents)
    return crossed

## The Main Program

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt


def main():

  X = []
  y = []
  for j in range(200):
    for k in range(200):
      x_coord = j / 20
      y_coord = k / 20
      X.append([x_coord, y_coord])

      # Classifing coordinates that lie above y = x/2
      if y_coord > (0.5) * x_coord:
          y.append(1)
      else:
          y.append(0)


  # Separate out data into training and testing data
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


  ga = GeneticAlgorithm()
  best_agent = XYAgent(0.01)

  num_generations = 10
  population_size = 10
  take_best = 1


  for i in range(0, num_generations):
    ga.populate(best_agent, population_size)
    evaluation = ga.evaluate(take_best, [X_train, y_train])
    print(evaluation)
    best_agent = ga.crossover(evaluation, XYAgent.crossover)


  print(best_agent.challenge(X_test, y_test))

  points = np.array([[i / 20, j / 20] for i in range(0, 201, 10) for j in range(0, 201, 10)])

  # Use the best agent to make predictions on these points
  predictions = [best_agent.brain(torch.tensor(point).float()).item() for point in points]
  colors = ['red' if pred >= 0.5 else 'blue' for pred in predictions]

    # Plot the points
  plt.figure(figsize=(8, 8))
  plt.scatter(points[:, 0], points[:, 1], c=colors)
  plt.title('Predictions of the Best Agent')
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.show()

if __name__ == "__main__":
    main()

0.0


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)